Note:  This notebook has widgets to select the state, congressional district and facility.  Those currently work only when the cells are run one at a time sequentially, not all at once.

## All Enforcements by Congressional District

This notebook examines ECHO data using CASE_FACILITIES and CASE_ENFORCEMENTS.
The CASE_NUMBER FIELD links these two tables.  Facilities within the congressional district are identified via the ECHO_EXPORTER data set, with the REGISTRY_ID of ECHO_EXPORTER matched against the same field of CASE_FACILITIES.
HQ_DIVISION identifies the program of the enforcement, and CASE_STATUS_DATE is taken to be the date the enforcement was concluded.


A state and congressional district must be chosen using the dropdown
widgets that are provided.

In [ ]:
# Import libraries
import urllib.parse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import folium

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display


Run this next cell, select a state, and then run the following cell.

In [ ]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
dropdown_state=widgets.Dropdown(
    options=states,
    value='NY',
    description='State:',
    disabled=False,
)
output_state = widgets.Output()
my_state = ""

def dropdown_state_eventhandler( change ):
    output_state.clear_output()
    value = change.new
    with output_state:
        display( change.new )
            
dropdown_state.observe( dropdown_state_eventhandler, names='value')
display( dropdown_state )

Run this next cell after choosing a state.

In [ ]:
my_state = dropdown_state.value

sql = "select REGISTRY_ID, FAC_NAME, FAC_STATE, FAC_LAT, FAC_LONG, AIR_IDS, FAC_DERIVED_CD113 " + \
    " from ECHO_EXPORTER where AIR_FLAG = 'Y' and FAC_STATE = '" + my_state + "'" 
url='http://apps.tlt.stonybrook.edu/echoepa/?query='
data_location=url+urllib.parse.quote(sql)
# print(sql)
# print(data_location)


In [ ]:
try:
    echo_data = pd.read_csv(data_location,encoding='iso-8859-1',header = 0)
    echo_data.set_index( "REGISTRY_ID", inplace=True)
except:
    print( "No data for " + my_state )

Run this next cell after choosing a state from the dropdown list.

In [ ]:
num_facilities = echo_data.shape[0]
print("There are %s facilities in %s tracked in the ECHO database." %(num_facilities, my_state))

Run this next cell to generate the Congressional District dropdown list for your state.  Then select a CD and run the following cell.
Here is a map of congressional districts: https://www.govtrack.us/congress/members/map

In [ ]:
if (( my_state != 'none' ) & (my_state != 'all' )):
    cd_array = echo_data["FAC_DERIVED_CD113"].fillna(0).astype(int).unique()
    cd_array.sort()
    w2=widgets.Dropdown(
        options=cd_array,
        value=1,
        description='Congressional Districts:',
        disabled=False,
    )
    display(w2)



Run this next cell after choosing a congressional district from the dropdown list.

In [ ]:
my_cd = w2.value
my_cd_facs = echo_data[echo_data["FAC_DERIVED_CD113"].fillna(0).astype(int) == my_cd]
num_facilities = my_cd_facs.shape[0]    
print("There are %s ICIS facilities in %s district %s tracked in the ECHO database." %(num_facilities, my_state, my_cd))

Next look up the violation history for the facilities in the selected
state and congressional district.  This step may take awhile.

In [ ]:
sql = "select cf.REGISTRY_ID, cf.CASE_NUMBER, ce.FISCAL_YEAR, ce.HQ_DIVISION, ce.TOTAL_PENALTY_ASSESSED_AMT " + \
    " from CASE_FACILITIES cf, CASE_ENFORCEMENTS ce where cf.CASE_NUMBER = ce.CASE_NUMBER " + \
    " and cf.STATE_CODE = '" + my_state + "'"

url='http://apps.tlt.stonybrook.edu/echoepa/?query='
data_location=url+urllib.parse.quote(sql)
print(sql)
print(data_location)

In [ ]:
sql = "select cf.REGISTRY_ID, cf.CASE_NUMBER, ce.FISCAL_YEAR, ce.HQ_DIVISION, ce.TOTAL_PENALTY_ASSESSED_AMT " + \
    " from CASE_FACILITIES cf, CASE_ENFORCEMENTS ce where cf.CASE_NUMBER = ce.CASE_NUMBER " + \
    " and cf.STATE_CODE = '" + my_state + "'"

url='http://apps.tlt.stonybrook.edu/echoepa/?query='
data_location=url+urllib.parse.quote(sql)

case_data = pd.read_csv(data_location,encoding='iso-8859-1',header = 0)
case_data.set_index( "REGISTRY_ID", inplace=True)



In [ ]:
case_fac_data = pd.merge( my_cd_facs, case_data, on='REGISTRY_ID' ).fillna( value={'HQ_DIVISION': 'Unknown'} )
case_facs = case_fac_data[['FAC_NAME', 'FAC_LAT', 'FAC_LONG']].drop_duplicates()
state_case_fac_data = pd.merge( echo_data, case_data, on='REGISTRY_ID' ).fillna( value={'HQ_DIVISION': 'Unknown'} )
state_case_facs = state_case_fac_data[['FAC_NAME', 'FAC_LAT', 'FAC_LONG']].drop_duplicates()

print( "There are %s facilities with %s EPA enforcements in the district." %(len(case_facs), len(case_fac_data)))
print( "There are %s facilities with %s EPA enforcements in the state." %(len(state_case_facs), len(state_case_fac_data)))


Save the data to a CSV file in your Google Drive.
The first of the next three cells will open our Google Drive to write into.
The second cell writes the congressional district file.
The third cell writes the file for state data.
Running these cells is optional.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Write the congressional district data to CSV file.

In [ ]:
filename = '/content/drive/My Drive/cd_cases-' + my_state + '-' + str( my_cd ) + '.csv'
case_fac_data.to_csv( filename ) 
print( "Writing this data to %s" %(filename))

Write the state data to CSV file.

In [ ]:
filename = '/content/drive/My Drive/state_cases-' + my_state + '.csv'
state_case_fac_data.to_csv( filename ) 
print( "Writing this data to %s" %(filename))

Map the facilities that have had these enforcement actions.

In [ ]:
# Let's show a quick map of your area and the facilities in it

def mapper(df):
    # Initialize the map
    m = folium.Map(
        location = [df.mean()["FAC_LAT"], df.mean()["FAC_LONG"]],
        zoom_start = 9
    )

    # Add a clickable marker for each facility
    for index, row in df.iterrows():
        folium.Marker(
            location = [row["FAC_LAT"], row["FAC_LONG"]],
            popup = row["FAC_NAME"] ).add_to(m)

    # Show the map
    return m

map_of_facilities_in_my_area = mapper(case_facs)
map_of_facilities_in_my_area

Present another dropdown with the list of names of the facilities.  Choosing a facility will display
the violations.

In [ ]:
cd_array = case_fac_data["FAC_NAME"].unique()
cd_array.sort()

w3=widgets.Dropdown(
    options=cd_array,
    description='Facility Name:',
    disabled=False,
)
display(w3)

Run this next cell after choosing a facility.

In [ ]:
my_cases = case_fac_data[case_fac_data["FAC_NAME"] == w3.value]
# my_cases[['FAC_NAME', 'FISCAL_YEAR', 'HQ_DIVISION', 'TOTAL_PENALTY_ASSESSED_AMT']]
# my_cases

## Plot the number of enforcements by year and program for the congressional district.

In [ ]:
chart_title = 'Number of enforcements by year in district'
case_fac_data.groupby(['FISCAL_YEAR','HQ_DIVISION']).size().unstack().plot(kind='bar', \
                                                                           title = chart_title, figsize=(15, 10), \
                                                                           legend=True, fontsize=12,stacked=True)
plt.show()

## Plot the number of enforcements by year and program for the state.

In [ ]:
chart_title = 'Number of enforcements in state by year'
state_case_fac_data.groupby(['FISCAL_YEAR','HQ_DIVISION']).size().unstack().plot(kind='bar', \
                                                                           title = chart_title, figsize=(15, 10), \
                                                                           legend=True, fontsize=12,stacked=True)
plt.show()